# Procesamiento de Datos Post-Imputación

Autores: Rafaella Ormond y Jose Jaime Martinez-Magana <br>
***Descripción:***<br>
Este script procesa datos imputados del servidor TOPMED, convirtiendo archivos VCF al formato PLINK y realizando filtrado de control de calidad.

**ENTRADA:** Archivos de salida de imputación del servidor TOPMed (por ejemplo, VCFs específicos por cromosoma o archivos de dosificación).<br>
**SALIDA:** Archivos de conjunto de datos PLINK con control de calidad, filtrados y listos para análisis posteriores.


### ***Requisitos:***
### Descargar Plink
Puedes descargar Plink versión 1.9 y versión 2.0 siguiendo los pasos de su sitio web.<br>
Para instalar plink2 [accede aquí](https://www.cog-genomics.org/plink/2.0/)<br>
Para instalar plink1.9 [accede aquí](https://www.cog-genomics.org/plink/1.9/) 

### Descargar BCFtools
**BCFtools** es otra herramienta popular para manejar archivos VCF/BCF:  
- [Descargar BCFtools](http://samtools.github.io/bcftools/)  

### Resultados de Imputación TOPMED
Este script asume que ya has completado la imputación usando el servidor TOPMED [enlace aquí](https://imputation.biodatacatalyst.nhlbi.nih.gov/#!) y descargado los resultados.<br>
Después de completar la imputación, deberás descargar:

- El **Informe de Control de Calidad** (por ejemplo, `qcreport.html`)
- Las **estadísticas resumen de QC** (por ejemplo, `chunks-excluded.txt`, `snps-excluded.txt`, `typed-only.txt`)
- Los **archivos VCF imputados**, uno por cromosoma (por ejemplo, `chr1.dose.vcf.gz`, `chr2.dose.vcf.gz`, ...)

Estos archivos están disponibles mediante enlaces directos desde la interfaz del servidor después de que tu trabajo finalice. Asegúrate de guardarlos en una estructura de carpetas consistente para el análisis posterior.

## Opciones de Software para QC

Puedes realizar el control de calidad post-imputación usando una de las siguientes opciones:

1) Plink
2) BCFtools

| Herramienta   | Pros ✅ | Contras ❌ |
|---------------|--------|-----------|
| **PLINK**     | - Ampliamente usado en pipelines de GWAS.<br>- Muy rápido para conjuntos de datos grandes.<br>- Muchas funciones integradas de QC genético (MAF, HWE, missingness, etc.).<br>- Puede generar múltiples formatos. | - Requiere conversión de VCF a formato PLINK primero.<br>- Manipulación limitada de estructuras VCF complejas comparado con BCFtools. |
| **BCFtools**  | - Trabaja directamente con archivos VCF/BCF comprimidos.<br>- Muy eficiente para filtrar y subestablecer archivos de variantes grandes.<br>- Flexible para automatización con scripts.<br>- Soporta streaming y piping con otras herramientas. | - Requiere más scripting manual para pasos de QC específicos de GWAS.<br>- Menos “listo para usar” para flujos de trabajo de asociación genética comparado con PLINK. |

---

💡 **Consejo:**  
- Si planeas avanzar rápidamente al análisis GWAS, **PLINK** suele ser la opción más sencilla.  
- Si quieres máxima flexibilidad y trabajas mucho con archivos VCF crudos, **BCFtools** es una opción potente.

Pasos del análisis:
1. **Descomprimir los archivos de imputación**<br>
2. **Plink**<br>
   2.1 Configuración del usuario<br>
   2.2 Crear archivos plink y unir todos los cromosomas<br>
   2.3 Unir archivos PLINK y estandarizar IDs de variantes<br>
   2.4 Realizar depuración de LD en el conjunto de datos PLINK unido

3. **BCFTools**<br>
   3.1 Configuración del usuario<br>
   3.2 Control de calidad<br>
   3.3 Convertir BCF filtrado a PLINK por cromosoma<br>
   3.4 Unir archivos PLINK específicos por cromosoma<br>
   3.5 Estandarizar IDs de variantes y conservar solo SNPs



### 1. Descomprimir los archivos de imputación
***Descripción:***<br>
Descomprime los archivos de salida de la imputación.

> **Nota:** Algunos servidores de imputación, como TOPMed, pueden cifrar los archivos de salida con una contraseña por motivos de seguridad.  
> La contraseña suele proporcionarse en la página de resultados de tu trabajo o en el correo de confirmación.  
> Asegúrate de reemplazar la contraseña de ejemplo por la específica de tu conjunto de datos.

Por favor ajusta la ruta y los archivos

In [ ]:
cd ${input_path}

# unzip all files
# add the password of your work in the next line
password='XXX'
for chr in *.zip
do
    echo unzip -P ${password} -o ${chr}
    unzip -P ${password} -o ${chr}
done

### 2. PLINK


### 2.1 Configuración del usuario

***Descripción:***<br>
Establece los parámetros y ajusta la ruta y los archivos según tu análisis

In [1]:
# input path
input_path="/path_to_your_data/imputation_results"

# set directory for files split by chromosome
splitted_path="/path_to_your_data/splitted_files"

# set the plink2 file
plink2="/path_to_your_data/plink2"

# set directory for files with the chromosomes merged
merged_path="/path_to_your_data/merged_files"

# set ld pruned path
ldpruned_path="/path_to_your_data/merged_files_ldpruned"


### 2.2 Crear archivos plink y unir todos los cromosomas
***Descripción:***<br>
Convierte los archivos `.dose.vcf.gz` al formato PLINK y une todos los cromosomas en un solo conjunto de datos.

Por favor ajusta la ruta y los archivos

In [ ]:
# transform dose files to plink
for chr in {1..22}
do
${plink2} --vcf ${input_path}/chr${chr}.dose.vcf.gz --make-pgen --id-delim '_' --rm-dup 'force-first' --snps-only --out ${splitted_path}/chr${chr}.dose.plink
done

### 2.3 Unir archivos PLINK y estandarizar IDs de variantes

***Descripción:*** <br>
Este script se mueve al directorio que contiene los archivos PLINK divididos por cromosoma, crea una lista de esos archivos, los une en un solo conjunto de datos y estandariza los IDs de variantes al formato chr:bp:ref:alt para consistencia en análisis posteriores.

In [ ]:
# move to splitted by chromosome path
cd ${splitted_path}

# create list of plink files if needed
# for chr in {2..22};do echo chr${chr}.dose.plink;done > plink_files.list

# load libraries
plink2="/path_to_your_data/plink2"

# transform dose files to plink
${plink2} --pfile chr1.dose.plink --pmerge-list plink_files.list pfile --multiallelics-already-joined --out ${merged_path}/merged.chr.dose.plink

# change all variants ID to chr:bp:ref:alt
${plink2} --pfile ${merged_path}/merged.chr.dose.plink --set-all-var-ids 'chr@:#:$1:$2' --snps-only 'just-acgt' --make-pgen --out ${merged_path}/merged.chr.dose.plink.norsids.snpsonly

### 3. BCFTools

### 3.1 Configuración del usuario

***Descripción:***<br>
Establece los parámetros y ajusta la ruta y los archivos según tu análisis

In [ ]:
input='/path_to_your_data/00impfiles'
filtered_path='/path_to_your_data/02impfilesfiltered'
plink_path="/path_to_your_data/plink2"
plink_out="/path_to_your_data/plink_per_chr"
merged_path="/path_to_your_data/merged_plink"

### 3.2 Control de calidad

**Descripción:**  
Este script procesa secuencialmente los archivos de variantes imputadas por cromosoma aplicando filtros de calidad usando `bcftools`. Las variantes se filtran según la calidad de imputación (R2) y la frecuencia alélica menor (MAF). Los archivos filtrados se comprimen e indexan, preparándolos para análisis genéticos posteriores.


In [ ]:
# filter BRHC imputed variants
# set the paths for input and output data

# loop over each chromosome sequentially
for chr in {1..22}; do
    bcftools view -i 'R2>0.8 & MAF>0.001' --threads 30 -Oz -o ${filtered_path}/cohort.chr${chr}.dose.filtered.vcf.gz ${input}/chr${chr}.dose.vcf.gz
    bcftools +fill-tags --threads 30 ${filtered_path}/cohort.chr${chr}.dose.filtered.vcf.gz --write-index -Ob -o ${filtered_path}/cohort.chr${chr}.dose.filtered.tag.bcf.gz -- -t AC,AN
done

### 3.3 Convertir BCF filtrado a PLINK por cromosoma
***Descripción:***<br> 
Convierte los archivos BCF filtrados al formato .pgen usando PLINK.<br>
Cambia "cohort" por el nombre del archivo

In [ ]:
for chr in {1..22}; do
plink2 \
  --bcf ${filtered_path}/cohort.chr${chr}.dose.filtered.tag.bcf.gz \
  --max-alleles 2 \
  --set-all-var-ids 'chr@:#:$1:$2' \
  --new-id-max-allele-len 100 \
  --make-pgen \
  --out ${plink_out}/cohort_chr${chr}.plink
done

### 3.4 Unir archivos PLINK específicos por cromosoma
***Descripción:*** 
Une los archivos `.pgen` individuales de cada cromosoma en un solo conjunto de datos PLINK.

Por favor modifica los nombres según tu archivo

In [ ]:
# move to de path
mkdir -p ${merged_path}

# Creat list of files to merge (exclude the chromosome 1, will be the base file)
for chr in {2..22}; do
    echo "${plink_out}/cohort_chr${chr}.plink" >> ${merged_path}/cohort_merge_list.txt
done

# Removing all duplicates
for chr in {1..22}
do
plink2 --pfile ${datapath}/cohort_chr${chr}.plink --rm-dup 'exclude-all' --make-pgen --out cohort_nodup_chr${chr} \
done

# Getting the file for merging
current_path= your_path
for chr in {2..22}
do
echo "${current_path}/cohort_nodup_chr${chr}" >> cohort_merging-09182025.txt
done

# Merging
plink2 --pfile cohort_nodup_chr1 --pmerge-list cohort_merging.txt 'pfile' --make-pgen --out cohort_nodup_merged

### 3.5 Estandarizar IDs de variantes y conservar solo SNPs
***Descripción:*** <br>
Actualiza los IDs de variantes al formato `@:#:$r:$a`, filtra para incluir solo SNPs A/C/G/T y recrea el archivo `.pgen`.

In [ ]:
${plink_path} \
    --pfile ${merged_path}/cohort_merged_allchr \
    --set-all-var-ids \
    --snps-only just-acgt \
    --make-pgen \
    --out ${merged_path}/cohort_merged_allchr.snpsonly